# Impermanent loss notes

In [1]:
# Simplified swap without fees
import math

# LIQUIDITY
# given some amount of an asset and pair reserves, returns an equivalent amount of the other asset
# used when adding liquidity
def quote(amountA, reserveA, reserveB):
    assert(amountA > 0)
    assert(reserveA > 0 and reserveB > 0)
    amountB = (amountA * reserveB) / reserveA
    return amountB

# SWAP
# simplified version of above
def get_amount_delta(amount0_delta, reserve0, reserve1):
    dx = amount0_delta
    x = reserve0
    y = reserve1
    return (-dx*y)/(x+dx)

# Simple ERC20 token
class SimpleToken:
    def __init__(self):
        self.balance_of = {}
        self.total_supply = 0
    def _mint(self, to, value):
        self.total_supply += value
        self.balance_of[to] = self.balance_of.get(to, 0) + value
    def _burn(self, from_, value):
        self.total_supply -= value
        self.balance_of[from_] = self.balance_of.get(from_, 0) - value
    
class SimpleSwap(SimpleToken):
    def __init__(self):
        super().__init__();
        self.reserve0 = 0
        self.reserve1 = 0
    def k(self):
        return self.reserve0 * self.reserve1
    def liquidity(self):
        return math.sqrt(self.k())
    
    def mint(self, to, amount0 = None, amount1 = None):
        assert(not (amount0 is None and amount1 is None));
        if amount0 is None:
            amount0 = quote(amount1, self.reserve1, self.reserve0)
        if amount1 is None:
            amount1 = quote(amount0, self.reserve0, self.reserve1)
            
        if self.total_supply == 0:
            # first mint
            liquidity = math.sqrt(amount0 * amount1)
            # omitted min. liquidity code for simplicity
        else :
            l1 = amount0/self.reserve0 * self.total_supply
            l2 = amount1/self.reserve1 * self.total_supply
            # ensures that amounts are added in same proportion otherwise the smallest percent wins?
            liquidity = Math.min(l1, l2)
        self.reserve0 += amount0
        self.reserve1 += amount1
        self._mint(to, liquidity)
        
    def swap(self, amount0_delta):
        # balance represented token reserve after a successful swap
        amount1_delta = get_amount_delta(amount0_delta, self.reserve0, self.reserve1)
        reserve0_new = self.reserve0 + amount0_delta
        reserve1_new = self.reserve1 + amount1_delta
        assert (reserve0_new * reserve1_new == self.k()), 'k invariant is not preserved'
        self.reserve0 = reserve0_new
        self.reserve1 = reserve1_new
    # Price of 1 token0 in token1
    def price0(self):
        return self.reserve1/self.reserve0
    # Price of 1 token1 in token0
    def price1(self):
        return self.reserve0/self.reserve1
    def reserves_priced_in_token0(self):
        return self.reserve0 + self.reserve1 * self.price1();
    def reserves_priced_in_token1(self):
        return self.reserve1 + self.reserve0 * self.price0();
    def pprint(self):
        print("k = {}".format(self.k()))
        print("lp_supply = {}".format(self.total_supply))
        print("reserve0 = {}".format(self.reserve0))
        print("reserve1 = {}".format(self.reserve1))
        print("total reserves = {} token0 aka {} token1".format(
            self.reserves_priced_in_token0(),
            self.reserves_priced_in_token1())
        )
        print("LPs = {}".format(self.balance_of))
        print("1 token0 = {} token1".format(self.price0()))
        print("1 token1 = {} token0".format(self.price1()))
    

In [2]:
swap = SimpleSwap()
swap.mint("0xbob", 1000, 20000)
swap.pprint()

k = 20000000
lp_supply = 4472.13595499958
reserve0 = 1000
reserve1 = 20000
total reserves = 2000.0 token0 aka 40000.0 token1
LPs = {'0xbob': 4472.13595499958}
1 token0 = 20.0 token1
1 token1 = 0.05 token0


In [3]:
swap.swap(100)
swap.pprint()

k = 20000000.0
lp_supply = 4472.13595499958
reserve0 = 1100
reserve1 = 18181.81818181818
total reserves = 2200.0 token0 aka 36363.63636363636 token1
LPs = {'0xbob': 4472.13595499958}
1 token0 = 16.52892561983471 token1
1 token1 = 0.060500000000000005 token0


In [4]:
swap.swap(-100)
swap.pprint()

k = 20000000.0
lp_supply = 4472.13595499958
reserve0 = 1000
reserve1 = 20000.0
total reserves = 2000.0 token0 aka 40000.0 token1
LPs = {'0xbob': 4472.13595499958}
1 token0 = 20.0 token1
1 token1 = 0.05 token0


In [5]:
def delta_x_for_price_x(desired_price_x, reserve0, reserve1):
    p = desired_price_x
    x = reserve0
    y = reserve1
    dx = math.sqrt(x*y/p) - x
    return dx
delta_x_for_price_x(16.52, 1000, 20000)

100.29712033915393

In [6]:
 def get_amount_in(amount_out, reserve_in, reserve_out):
    numerator = reserve_in * amount_out
    denominator = reserve_out - amount_out
    amount_in = (numerator / denominator)
    return amount_in

print(get_amount_delta(-100, 1000, 20000))
print(get_amount_in(100, 20000, 1000))

2222.222222222222
2222.222222222222


In [7]:
s = SimpleSwap()
s.mint("0xbob", 100, 200)
s.pprint()

k = 20000
lp_supply = 141.4213562373095
reserve0 = 100
reserve1 = 200
total reserves = 200.0 token0 aka 400.0 token1
LPs = {'0xbob': 141.4213562373095}
1 token0 = 2.0 token1
1 token1 = 0.5 token0


In [8]:
delta_x = delta_x_for_price_x(1, s.reserve0, s.reserve1)
delta_x

41.42135623730951

In [9]:
s.swap(delta_x)
s.pprint()

k = 20000.0
lp_supply = 141.4213562373095
reserve0 = 141.4213562373095
reserve1 = 141.4213562373095
total reserves = 282.842712474619 token0 aka 282.842712474619 token1
LPs = {'0xbob': 141.4213562373095}
1 token0 = 1.0 token1
1 token1 = 1.0 token0


In [10]:
s2 = SimpleSwap()
s2.mint("0xbob", 100, 200)
s2.pprint()

k = 20000
lp_supply = 141.4213562373095
reserve0 = 100
reserve1 = 200
total reserves = 200.0 token0 aka 400.0 token1
LPs = {'0xbob': 141.4213562373095}
1 token0 = 2.0 token1
1 token1 = 0.5 token0


In [11]:
s2.swap(delta_x_for_price_x(4, s2.reserve0, s2.reserve1))
s2.pprint()

k = 20000.0
lp_supply = 141.4213562373095
reserve0 = 70.71067811865476
reserve1 = 282.842712474619
total reserves = 141.4213562373095 token0 aka 565.685424949238 token1
LPs = {'0xbob': 141.4213562373095}
1 token0 = 4.0 token1
1 token1 = 0.25 token0


In [12]:
hodl = 100 * 4 + 200 # in token1
hodl

600

In [13]:
lp = s2.reserve0 * 4 + s2.reserve1 # token1
lp

565.685424949238

In [14]:
hodl = 100 + 200 * 0.25 # in token0
hodl

150.0

In [15]:
lp = s2.reserve0 + s2.reserve1 * 0.25 # in token0
lp

141.4213562373095

In [16]:
s3 = SimpleSwap()
s3.mint("0xbob", 100, 200)
s3.pprint()

k = 20000
lp_supply = 141.4213562373095
reserve0 = 100
reserve1 = 200
total reserves = 200.0 token0 aka 400.0 token1
LPs = {'0xbob': 141.4213562373095}
1 token0 = 2.0 token1
1 token1 = 0.5 token0


In [17]:
dx = delta_x_for_price_x(1.5, s3.reserve0, s3.reserve1)
print(dx)
s3.swap(dx)
s3.pprint()

15.47005383792515
k = 20000.0
lp_supply = 141.4213562373095
reserve0 = 115.47005383792515
reserve1 = 173.20508075688772
total reserves = 230.9401076758503 token0 aka 346.41016151377545 token1
LPs = {'0xbob': 141.4213562373095}
1 token0 = 1.5 token1
1 token1 = 0.6666666666666666 token0


In [18]:
hodl = 100 * 1.5 + 200 # token1
hodl

350.0

In [19]:
lp = s3.reserve0 * 1.5 + s3.reserve1 # token1
lp

346.41016151377545